## 🦜🔗 Langgraph Checkpoints und Agenten, die vor Toolbenutzung nachfragen


Manchmal möchte man, dass ein Agent etwas tun darf, aber nur nach Rückfrage. Typisch Anwendungsfälle sind z.B. Email-Versand oder der Zugriff auf das Betriebssystem.

In diesem Notebook wollen wir zwei Möglichkeiten untersuchen, dies zu tun.

- Naiver Ansatz. Wir bauen ein Terminalprompt ein. Und zwar im Codefluss genau vor der Stelle, wo die Anwendung kritische Berechtigungen braucht.
- Lösung mit LangGraph-Checkpoints. Dieser Teil ist z.T. sehr detailreich. Man muss sich wirklich nicht alles davon merken.

### Lösung 1. Mit einem Terminalprompt


In [ ]:
from langchain import hub
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation
from langchain.agents import create_tool_calling_agent
from langchain.agents.output_parsers.tools import ToolAgentAction
from langchain.tools.shell import ShellTool
from helpers import llm

tools = [ShellTool()]
prompt = hub.pull("reactagent/openai-functions-agent")
agent_runnable = create_tool_calling_agent(llm(temperature=0), tools, prompt)
tool_executor = ToolExecutor(tools)

In [ ]:
from typing import TypedDict, Annotated, Union
from langchain_core.agents import AgentActionMessageLog, AgentFinish
import operator


class AgentState(TypedDict):
    input: str
    agent_outcome: Union[AgentActionMessageLog, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentActionMessageLog, str]], operator.add]

In [ ]:
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}


def execute_tools(data):
    agent_actions: AgentActionMessageLog = data["agent_outcome"]
    outputs = []
    for agent_action in agent_actions:
        if agent_action.tool == "terminal":
            response = input(
                prompt=f"[y/n] continue with shell execution: {agent_action.tool_input}?"
            )
            if response == "y":
                output = tool_executor.invoke(agent_action)
            else:
                output = "Your terminal command was not permitted by the user. Try a different terminal command or return unfinished."
            outputs.append((agent_action, str(output)))
        else: outputs.append((agent_action, tool_executor.invoke(agent_action)))
    print("outputs", outputs)
    return {"intermediate_steps": outputs}
    # return {"intermediate_steps": [(agent_actions, str(output))]}


def should_continue(data):
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "continue"

In [ ]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)
workflow.add_edge("action", "agent")

chain = workflow.compile()

#### Wir definieren eine kleine Funktion, die die Ausgabe formatiert.

Diese Arbeit muss man üblicherweise selbst tun, weil LangChain ja nicht weiß, welches Format man am Ende braucht.


In [ ]:
def formatted_output(data):
    print('data', data)
    int_steps = data.get("intermediate_steps")
    for step in int_steps:
        print(step)

    return data.get("agent_outcome").return_values.get("output")


app = chain | formatted_output

In [ ]:
inputs = {
    "input": "Count the lines of all python notebooks in the current directory. Use simple shell commands."
}
output = app.invoke(inputs)
print(f"Agent result: {output}")

Das hat funktioniert

Allerdings muss nun unsere App den GraphState und alle Objekte so lange im Memory behalten, bis ein Nutzer endlich die Rückfrage beantwortet. Asynchron ist das ganz schön blöd.


### Lösung 2. LangGraph- Checkpoints

Checkpoints sind ein essentieller Baustein von Langgraph. Bis jetzt haben wir noch nichts davon mitbekommen. Was tun Checkpoints und wozu brauchen wir die?

Die LangChain-Doku ist leider sehr dünn, was CheckPoints angeht und für die meisten Demo-Zwecke braucht man Checkpoints auch nicht.

Weil das Konzept für eine reale App mit realen Nutzern sehr schnell relevant wird, erläutern wir es hier grob.

Wenn eine LangGraph-App mit einem Knoten fertig ist und nachschaut, wohin sie jetzt weiterhüpft (zu welchem Knoten), speichert sie erst einmal den State (und noch andere Dinge) in einen Checkpoint. Das passiert alles in einem kleinen Memory-Objekt und muss uns nicht weiter interessieren. Der nächste Knoten liest dann aus dem Checkpoint des letzten Knoten aus und setzt daran an.

Wir können diesen Prozess explizit machen, indem wir den Checkpoint nicht in Memory speichern, sondern z.B. in einer Datenbank ablegen. Redis bietet sich hier an, weil Redis ja bekanntlich sehr schnell ist. Wir wollen aber keinen extra Redis-Container, also benutzen wir Sqlite.

Was bringt das?

- Nun kann ein Nutzer eine App, die ihm zu lange braucht, terminieren. Bisher abgelaufene Zwischenstände gehen nicht verloren. Er kann die App dann entweder neu starten oder auf dem letzten Zwischenstand aufsetzen.
- Man kann der App befehlen, vor einem bestimmten Knoten immer zu terminieren. Der Zwischenstand speichert sich automatisch und der Nutzer muss dann die App erneut dort aufrufen (Das bauen wir jetzt).
- Man kann verteilte Systeme bauen, in denen Komponenten ihren Arbeitstand untereinander mittels der Datenbank austauschen.
- Man kann die Checkpoints auch für die gesamte Chathistory eines Chatbots verwenden. Das sollte man mit Bedacht tun, ist aber prinzipiell möglich.


In [ ]:

#TODO: prüfen ob weiter oben auch mit .batch gearbeitet werden kann anstatt der for schleife um termin in tool calls zu finden
#TODO: prüfen warum weiter unten der code umfällt 

import json
from typing import List


# def unsafe_execute_tools(data):
#     actions: list[ToolInvocation] = []
#     last_message = data["agent_outcome"]
#     print('last message', last_message)

#     for tool_call in last_message:
#         actions.append(
#             ToolInvocation(
#                     tool=tool_call["name"],
#                     tool_input=tool_call["args"],
#                 )
#         )

#     responses = tool_executor.batch(actions)

#     tool_messages = [
#             ToolMessage(
#                 tool_call_id=tool_call["id"],
#                 name=tool_call["name"],
#                 content=response,
#             )
#             for tool_call, response in zip(last_message.tool_calls, responses)
#         ]
#     return tool_messages


# def unsafe_execute_tools(data):
#     agent_actions: List[AgentActionMessageLog] = data["agent_outcome"][-1]
#     print('agent_actions', agent_actions)
#     output = tool_executor.invoke(agent_actions)
#     print('output1', output)

#     return {"intermediate_steps": [(agent_actions, str(output))]}

# for tool_call in last_message.additional_kwargs["tool_calls"]:
#         function = tool_call["function"]
#         function_name = function["name"]
#         _tool_input = json.loads(function["arguments"] or "{}")
#         actions.append(
#             ToolInvocation(
#                 tool=tool_name,
#                 tool_input=_tool_input,
#             )
#         )


### WORKING
def unsafe_execute_tools(data):
    agent_actions: List[AgentActionMessageLog] = data["agent_outcome"][-1]
    print('agent_actions', agent_actions)
    output = tool_executor.invoke(agent_actions)
    print('output1', output)

    return {"intermediate_steps": [(agent_actions, str(output))]}

#### Jetzt kompilieren wir erneut

diesmal mit

- unsafe_execute_tools
- Checkpointer
- Interrupt vor der "action"-Node


In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", unsafe_execute_tools)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)
workflow.add_edge("action", "agent")

checkpoint_agent_executor = workflow.compile(
    checkpointer=memory, interrupt_before=["action"]
)

In [ ]:
inputs = {
    "input": "Count the lines of all python notebooks in the current directory. Use simple shell commands."
}
config = {"configurable": {"thread_id": "2"}}
for event in checkpoint_agent_executor.stream(inputs, config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

#### Aha

Hat terminiert. Eine kleine Hilfsfunktion testet den Stand unserer App:


In [ ]:
from helpers import is_resumeable

is_resumeable(checkpoint_agent_executor, config)

In [ ]:
inputs = None
# for event in checkpoint_agent_executor.stream(inputs, config):
#     for k, v in event.items():
#         if k != "__end__":
#             print(v)‘

print(checkpoint_agent_executor.invoke(inputs, config))

In [ ]:
is_resumeable(checkpoint_agent_executor, config)

#### LangGraph debugging

Wir kompilieren noch ein letztes mal mit debug=True um dieses Feature auch gesehen zu haben


In [ ]:
memory = SqliteSaver.from_conn_string(":memory:")
checkpoint_agent_executor = workflow.compile(
    checkpointer=memory, debug=True, interrupt_before=["action"]
)
inputs = {
    "input": "Count the the words in the README.md file in the current directory. Use simple shell commands."
}
for event in checkpoint_agent_executor.stream(inputs, config):
    pass

In [ ]:
inputs = None
for event in checkpoint_agent_executor.stream(inputs, config):
    pass